In [1]:
import cv2
from tensorflow.keras.models import load_model

In [2]:
cap = cv2.VideoCapture(0)
model = load_model('my_model.h5')

In [3]:
while True:
    _, frame = cap.read()
    cv2.rectangle(frame, (0, 0), (128, 128), (255, 255, 255), 2)
    cv2.imshow("data", frame)
    cv2.imshow("ROI", frame[:128, :128])
    img = cv2.cvtColor(frame[:128, :128], cv2.COLOR_RGB2GRAY)
    img = img.reshape(1, 128, 128, 1).astype("float32")
    z_predict = model.predict(img)
    print(z_predict)
    if ord('q') == 0xFF & cv2.waitKey(1):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 7s 7s/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 46ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 39ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 42ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 42ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 47ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 51ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 42ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 41ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 45ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 82ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
1/1 [==============================] - 0s 83ms/step
[[0.